In [1]:
# this is the main script, to analyze imput images via yolo, pass the resluts to more specific classificators

In [2]:
import custom_utils as cu
from ultralytics import YOLO
import cv2
import tensorflow as tf
from keras.models import load_model
import numpy as np

In [3]:
import os
from dotenv import load_dotenv
# Load .env files
load_dotenv()

# Get environment variables
dataset_dir = os.getenv('DATASET_DIR')
input_data = os.path.join(dataset_dir, 'test_data')

In [4]:
# Load images
image_paths,images = cu.get_images(input_data)

In [5]:
# Load a model(s)
model = YOLO('yolov8n.pt') 

In [6]:
# Run batched inference on a list of images
# results get saved in runs/detect folder 
results = model.predict(images, save=True, save_txt = True)  # return a list of Results objects


0: 640x640 2 persons, 2 bicycles, 3 cars, 3 traffic lights, 1 stop sign, 1: 640x640 10 cars, 1 bus, 5 trucks, 2 traffic lights, 1 stop sign, 410.0ms
Speed: 5.6ms preprocess, 205.0ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict9
2 labels saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict9/labels


In [ ]:
res = cu.extract_boxes(results, 9, target_size=(74,147))
res = np.array([cv2.cvtColor(r, cv2.COLOR_BGR2RGB) for r in res])

In [ ]:
# plot image for debug not necessary 
i = 0
while i < len(res):
    cv2.imshow("Hi", res[i])
    key = cv2.waitKey(0)

    if key == ord("0"):
        break
    elif key == ord("1"):
        i += 1
            
    cv2.destroyAllWindows()

In [13]:
# should be append by traffic sign
light_model = load_model("traffic_light_model.keras")

2023-08-14 23:37:28.662415: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [14]:
predicted = light_model.predict(res)

1/1 [==============================] - 0s 218ms/step


In [ ]:
# Out put traffic light detection
print(predicted)
i = 0
labels = ["green", "none", "red", "yellow"]
while i < predicted.shape[0]:
    cv2.imshow(labels[np.argmax(predicted[i])], res[i])
    key = cv2.waitKey(0)

    if key == ord("0"):
        break
    elif key == ord("1"):
        i += 1
            
    cv2.destroyAllWindows()